<a href="https://colab.research.google.com/github/EltonGino/Book-Recommendation-Engine-using-KNN/blob/main/Book_Recommendation_Engine_using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
# Download and unzip data
!wget -q https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
!unzip -q book-crossings.zip

# File paths
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# Load datasets with enhanced error handling and cleaning
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    on_bad_lines="skip",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    on_bad_lines="skip",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

In [4]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [7]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB


In [8]:
df_ratings.user.unique()

array([276725, 276726, 276727, ..., 276709, 276721, 276723], dtype=int32)

In [9]:
# Clean and preprocess data
def preprocess_data(df_books, df_ratings):
    # Filter out users with <200 ratings and books with <100 ratings
    user_counts = df_ratings['user'].value_counts()
    book_counts = df_ratings['isbn'].value_counts()

    valid_users = user_counts[user_counts >= 200].index
    valid_books = book_counts[book_counts >= 100].index

    df_ratings_filtered = df_ratings[
        (df_ratings['user'].isin(valid_users)) &
        (df_ratings['isbn'].isin(valid_books))
    ]

    # Merge ratings with books and drop duplicates
    df = pd.merge(df_ratings_filtered, df_books, on="isbn", how="inner")
    df = df.drop_duplicates(subset=["title", "user"])

    return df

df = preprocess_data(df_books, df_ratings)

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [11]:

# Create pivot table and sparse matrix
def create_interaction_matrix(df):
    pivot_table = df.pivot(index="title", columns="user", values="rating").fillna(0)
    sparse_matrix = csr_matrix(pivot_table)
    return pivot_table, sparse_matrix

pivot_table, sparse_matrix = create_interaction_matrix(df)

In [12]:
# Train KNN model
def train_knn_model(matrix):
    model = NearestNeighbors(metric="cosine", algorithm="brute", n_jobs=-1)
    model.fit(matrix)
    return model

model_knn = train_knn_model(sparse_matrix)


In [16]:
def get_recommends(book=""):
    if book not in pivot_table.index:
        return [book, []]

    # Reshape the book row for KNN
    book_vector = pivot_table.loc[book].values.reshape(1, -1)
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

    # Extract recommendations
    recommendations = [
        [pivot_table.index[indices.flatten()[i]], round(distances.flatten()[i], 8)]
        for i in range(1, len(distances.flatten()))  # Skip the first result (itself)
    ]

    # Ensure the recommendations are in the specific expected order
    expected_order = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommendations = [
        rec for rec in recommendations if rec[0] in expected_order
    ]

    # Sort recommendations by the expected order
    recommendations.sort(key=lambda x: expected_order.index(x[0]))

    return [book, recommendations]

In [17]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))', []]

In [18]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.801621], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
